In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model,layers
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger

import os

#traindata = pd.read_csv('preprocessed_train_multiclass(a)_new.csv', header=None)
#testdata = pd.read_csv('preprocessed_test_multiclass(a)_new.csv', header=None)
'''
X=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None )
T=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None )
C=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None)
Y=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None)
X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\train_label.csv", header=None)

T = pd.read_csv(r"C:\e\学习资料\02-03\pva1\pva_1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_label.csv", header=None)'''
f1 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\benign\benign1_b.csv", header=None)
f2 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\benign\benign2_b.csv", header=None)
x2 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\2\split2new_s3b.csv", header=None)
X = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\split1new_s3b.csv", header=None)

y2 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\2\split2news3_benign.csv",header=None)
Y = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\split1news3_benign.csv",header=None)
Y = Y.append ( y2 )

X = X.append (f1)
X = X.append (x2)
X = X.append (f2)
T = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3new_s3b.csv", header=None)
C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\3\split3news3_benign.csv",header=None)
f3 = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\benign\benign3_b.csv", header=None)
T = T.append (f3)
X1 = X

T1 = T
X1.fillna(0,inplace=True)
X1.replace([np.inf,-np.inf],1,inplace=True)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
#X1.value_counts('Label')
'''
for index in X1.columns:
    X1[index]=X1[index].fillna(0,inplace=True)
    X1[index]=X1[index].replace([np.inf,-np.inf],1,inplace=True)
'''

#scaler = Normalizer().fit(X1)
#trainX = scaler.transform(X1)
trainX =  np.array(X1)

#scaler = Normalizer().fit(T1)
#testT = scaler.transform(T1)
testT = np.array(T1)
y_train1 = np.array(Y)
y_test1 = np.array(C)

#y_train = to_categorical(y_train1)
#y_test = to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
#X_train = trainX
#X_test = testT

print (trainX.shape )
X_train = np.reshape(trainX, (trainX.shape[0],32,24))
X_test = np.reshape(testT, (testT.shape[0],32,24))

# Add a channels dimension
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
batch_size = 50

print (X_train.shape )

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train1)).shuffle(1000000).batch(50)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test1)).batch(50)


class MyModel(Model):
    #
    def __init__(self):
        #
        super(MyModel, self).__init__()
        '''
        self.lstm1=LSTM(50,input_shape=(1, 768), return_sequences= True )
        self.drop1=Dropout(0.01)
        self.lstm2=LSTM(50,return_sequences=True)
        self.drop2=Dropout(0.01)
        self.lstm3=LSTM(50, return_sequences=False)
        self.drop3=Dropout(0.01)
        '''
        self.flatten0=layers.Flatten(input_shape=(768,1))
        self.d0=layers.Dense(units=64,activation='relu',)
        self.cov1=layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 24, 1))
        self.pl1=layers.MaxPooling2D((2, 2))
       
        self.cov2=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl2=layers.MaxPooling2D((2, 2))
        self.cov3=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl3=layers.MaxPooling2D((2, 2))
        #self.flatten=layers.Flatten()
        #self.d0=layers.Dense(64, activation='relu')
        #self.drop1=Dropout(0.5)
        self.cov4=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.pl4=layers.MaxPooling2D((2, 2))
        self.cov5=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.flatten1=layers.Flatten()
        self.d1=layers.Dense(64, activation='relu')
        self.drop2=Dropout(0.5)
        self.d2=Dense(10)
        self.activation1=Activation('softmax')

    def call(self, x):
        #
        #x = self.flatten1(x)
        
        x = self.cov1(x)
        x = self.pl1(x)
        x = self.cov2(x)
        x = self.pl2(x)
        x = self.cov3(x)
       
       
       # x = self.pl3(x)
       # x = self.flatten(x)
       # x = self.d0(x)
       # x = self.drop1(x)
       # x = self.cov4(x)
       # x = self.pl4(x)
       # x = self.cov5(x)
        x = self.flatten1(x)
        # x = self.flatten0(x)
        
        x = self.d0(x)
        x = self.d1(x)
        x = self.drop2(x)
      
        x = self.d2(x)
        return self.activation1(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.CategoricalCrossentropy()
#lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')


@tf.function
def train_step(traind, labels):
    
    with tf.GradientTape() as tape:
        #
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(traind, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 30

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

#!mkdir -p saved_model
model.save('saved_model/my_modelb')
'''
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)
'''


(715792, 768)
(715792, 32, 24, 1)
Epoch 1, Loss: 0.32029780745506287, Accuracy: 86.01353454589844, Test Loss: 0.22510765492916107, Test Accuracy: 88.8635025024414
Epoch 2, Loss: 0.2462945431470871, Accuracy: 88.23401641845703, Test Loss: 0.21374666690826416, Test Accuracy: 89.08199310302734
Epoch 3, Loss: 0.23413635790348053, Accuracy: 88.69377899169922, Test Loss: 0.21773454546928406, Test Accuracy: 89.47636413574219
Epoch 4, Loss: 0.22896166145801544, Accuracy: 88.9903793334961, Test Loss: 0.21767710149288177, Test Accuracy: 89.56153869628906
Epoch 5, Loss: 0.2248668223619461, Accuracy: 89.13916015625, Test Loss: 0.20881661772727966, Test Accuracy: 89.61463928222656
Epoch 6, Loss: 0.2232750505208969, Accuracy: 89.21977233886719, Test Loss: 0.2071658968925476, Test Accuracy: 89.7219467163086
Epoch 7, Loss: 0.2217082381248474, Accuracy: 89.31198120117188, Test Loss: 0.2113029509782791, Test Accuracy: 89.47470092773438
Epoch 8, Loss: 0.22075530886650085, Accuracy: 89.31044006347656, Tes

"\nnew_model = tf.keras.models.load_model('saved_model/my_model')\n\n# Check its architecture\nnew_model.summary()\n# Evaluate the restored model\nloss, acc = new_model.evaluate(test_images, test_labels, verbose=2)\nprint('Restored model, accuracy: {:5.2f}%'.format(100 * acc))\n\nprint(new_model.predict(test_images).shape)\n"

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model,layers
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger

import os

#traindata = pd.read_csv('preprocessed_train_multiclass(a)_new.csv', header=None)
#testdata = pd.read_csv('preprocessed_test_multiclass(a)_new.csv', header=None)
'''
X=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None )
T=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None )
C=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None)
Y=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None)
X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\train_label.csv", header=None)

T = pd.read_csv(r"C:\e\学习资料\02-03\pva1\pva_1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_label.csv", header=None)'''
'''
X = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\sp1\d3\split1_d1_bert.csv", header=None)
Y = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\sp1\d3\split1_d1_bert_label.csv",header=None)

T = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\sp1\d3\split1_d1_bert.csv", header=None)
C = pd.read_csv(r"C:\e\学习资料\02-03\data\data_process\sp1\d3\split1_d1_bert_label.csv", header=None)
'''
f1 = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\sptrain1_f1_bert.csv", header=None)
f2 = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\sptrain1_f2_bert.csv", header=None)
f1_label = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\f1_label.csv", header=None)
f2_label = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\f2_label.csv", header=None)

X = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\sptrain1_t1_bert.csv", header=None)
Y = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\t1_label.csv", header=None)
X = X.append ( f1 )
X = X.append ( f2 )
Y = Y.append ( f1_label )
Y = Y.append ( f2_label )

T = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\sptrain1_t2_bert.csv", header=None)
C = pd.read_csv(r"C:\e\学习资料\02-03\ptrain1\tr1\t\t2_label.csv", header=None)



X1 = X
T1 = T
X1.fillna(0,inplace=True)
X1.replace([np.inf,-np.inf],1,inplace=True)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
#X1.value_counts('Label')
'''
for index in X1.columns:
    X1[index]=X1[index].fillna(0,inplace=True)
    X1[index]=X1[index].replace([np.inf,-np.inf],1,inplace=True)
'''

#scaler = Normalizer().fit(X1)
#trainX = scaler.transform(X1)
trainX =  np.array(X1)

#scaler = Normalizer().fit(T1)
#testT = scaler.transform(T1)
testT = np.array(T1)
y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train = to_categorical(y_train1)
y_test = to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
X_train = trainX
X_test = testT

print (trainX.shape )
X_train = np.reshape(trainX, (trainX.shape[0],32,24))
X_test = np.reshape(testT, (testT.shape[0],32,24))

# Add a channels dimension
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
batch_size = 50

print (X_train.shape )

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train1)).shuffle(1000000).batch(50)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test1)).batch(50)


class MyModel(Model):
    #
    def __init__(self):
        #
        super(MyModel, self).__init__()
        '''
        self.lstm1=LSTM(50,input_shape=(1, 768), return_sequences= True )
        self.drop1=Dropout(0.01)
        self.lstm2=LSTM(50,return_sequences=True)
        self.drop2=Dropout(0.01)
        self.lstm3=LSTM(50, return_sequences=False)
        self.drop3=Dropout(0.01)
        '''
        self.flatten0=layers.Flatten(input_shape=(768,1))
        self.d0=layers.Dense(units=64,activation='relu',)
        self.cov1=layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 24, 1))
        self.pl1=layers.MaxPooling2D((2, 2))
       
        self.cov2=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl2=layers.MaxPooling2D((2, 2))
        self.cov3=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl3=layers.MaxPooling2D((2, 2))
        #self.flatten=layers.Flatten()
        #self.d0=layers.Dense(64, activation='relu')
        #self.drop1=Dropout(0.5)
        self.cov4=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.pl4=layers.MaxPooling2D((2, 2))
        self.cov5=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.flatten1=layers.Flatten()
        self.d1=layers.Dense(64, activation='relu')
        self.drop2=Dropout(0.5)
        self.d2=Dense(9)
        self.activation1=Activation('softmax')

    def call(self, x):
        #
        
        x = self.cov1(x)
        x = self.pl1(x)
        x = self.cov2(x)
        x = self.pl2(x)
        x = self.cov3(x)
       
       # x = self.pl3(x)
       # x = self.flatten(x)
       # x = self.d0(x)
       # x = self.drop1(x)
       # x = self.cov4(x)
       # x = self.pl4(x)
       # x = self.cov5(x)
        x = self.flatten1(x)
       # x = self.flatten0(x)
        x = self.d0(x)
        x = self.d1(x)
        x = self.drop2(x)
        x = self.d2(x)
        return self.activation1(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')


@tf.function
def train_step(traind, labels):
    
    with tf.GradientTape() as tape:
        #
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(traind, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 20

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

#!mkdir -p saved_model
#model.save('saved_model/temp/bert_model')
'''
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)
'''


In [7]:
print ( Y.value_counts())

0    1    2    3    4    5    6    7    8  
1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    67530
0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     7506
          0.0  0.0  1.0  0.0  0.0  0.0  0.0     4986
     1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     3142
     0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     1874
                                   0.0  1.0     1506
                              1.0  0.0  0.0      636
               1.0  0.0  0.0  0.0  0.0  0.0      458
               0.0  0.0  1.0  0.0  0.0  0.0      324
dtype: int64
